In [ ]:
#module importing
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time

# 2. driver경로설정 및 url주소 설정
driverpath = 'C:/Users/Heewon/driver/chromedriver.exe'
dr = webdriver.Chrome(driverpath)

####

#로그인
dr.get('https://www.kita.net/login/login.do?geturl=http://stat.kita.net/stat/pmsd/china/ChinaWholeList.screen')
dr.find_element_by_id('p_member_id').send_keys('kedkorea12')
dr.find_element_by_id('p_passwd').send_keys('ked2363!')
dr.find_element_by_xpath('''//*[@id="myform"]/fieldset/button''').click()

####

df_tot = pd.DataFrame(columns = ['year','month','country','SC','S_code', 'export','export_rate', 'import','import_rate','CA'])

#한국 URL
naraURL = ['http://stat.kita.net/stat/kts/pum/ItemImpExpList.screen']
nara = ['한국']
for site, 나라 in zip(naraURL, nara) :
    #사이트 진입
    dr.get(site)
    #누계 -> 당월
    dr.find_element_by_xpath('''//*[@id="contents"]/div[3]/form/fieldset/div[2]/div[4]/select/option[1]''').click()
    #100개씩 보기
    dr.find_element_by_xpath('''//*[@id="listCount"]/option[3]''').click()
    
    #연도 43개
    for year_ in range(10,20) : 
        dr.find_element_by_xpath('''//*[@id="contents"]/div[3]/form/fieldset/div[2]/div[1]/select/option[{0}]'''.format(year_)).click()
        년 = int(dr.find_element_by_xpath('''//*[@id="contents"]/div[3]/form/fieldset/div[2]/div[1]/select/option[{0}]'''.format(year_)).text)
        
        #월 12개
        for month_ in range(1,13) : 
            
            if ((년 == 2019) and (month_ == 2)) :
                break;
            dr.find_element_by_xpath('''//*[@id="contents"]/div[3]/form/fieldset/div[2]/div[2]/select/option[{0}]'''.format(month_)).click()
            월 = int(dr.find_element_by_xpath('''//*[@id="contents"]/div[3]/form/fieldset/div[2]/div[2]/select/option[{0}]'''.format(month_)).text)

                
            #조회
            dr.find_element_by_xpath('''//*[@id="contents"]/div[3]/form/fieldset/div[3]/a''').click()                    
            time.sleep(0.5)
            
            while 1:
                try :
                    drt = dr.page_source
                    soup = BeautifulSoup(drt,'html.parser')

                    #데이터 적재
                    코드_소 = []
                    소분류 = []
                    수출액 = []
                    수출증감률 = []
                    수입액 = []
                    수입증감률 = []
                    수지 = []

                    #첫페이지 
                    #데이터 적재
                    i=1
                    for cycle in range(int((len(soup.findAll('td', {'align' : 'Left'}))-1)/2)) :
                        코드_소.append(soup.findAll('td', {'align' : 'Left'})[-i+1].text)
                        소분류.append(soup.findAll('td', {'align' : 'Left'})[i*2].text)
                        수출액.append(soup.findAll('td', {'align' : 'Right'})[i*10+5].text)
                        수출증감률.append(soup.findAll('td', {'align' : 'Right'})[i*10+6].text)
                        수입액.append(soup.findAll('td', {'align' : 'Right'})[i*10+7].text)
                        수입증감률.append(soup.findAll('td', {'align' : 'Right'})[i*10+8].text)
                        수지.append(soup.findAll('td', {'align' : 'Right'})[i*10+9].text)
                        i+=1

                        df = pd.DataFrame({'year' :년, 'month' :월, 'country':나라, 'SC':소분류, 'S_code':코드_소, 'export':수출액, 'export_rate':수출증감률, 'import':수입액, 'import_rate':수입증감률, 'CA':수지})
                        df_tot = pd.concat([df_tot, df])

                    ####################
                    for tmp in range(1, (len(soup.find('div', class_='boardnavi').find('span').findAll('a'))+1)):

                        print(dr.find_element_by_xpath('''//*[@id="pageArea"]/span/a[{0}]'''.format(tmp)).text)
                        dr.find_element_by_xpath('''//*[@id="pageArea"]/span/a[{0}]'''.format(tmp)).click()
                        time.sleep(0.5)
                        tmp += 1
                        drt = dr.page_source
                        soup = BeautifulSoup(drt,'html.parser')

                        #데이터 적재
                        코드_소 = []
                        소분류 = []
                        수출액 = []
                        수출증감률 = []
                        수입액 = []
                        수입증감률 = []
                        수지 = []
                        #데이터 적재
                        i=1
                        for cycle in range(int((len(soup.findAll('td', {'align' : 'Left'}))-1)/2)):
                            코드_소.append(soup.findAll('td', {'align' : 'Left'})[-i+1].text)
                            소분류.append(soup.findAll('td', {'align' : 'Left'})[i*2].text)
                            수출액.append(soup.findAll('td', {'align' : 'Right'})[i*10+5].text)
                            수출증감률.append(soup.findAll('td', {'align' : 'Right'})[i*10+6].text)
                            수입액.append(soup.findAll('td', {'align' : 'Right'})[i*10+7].text)
                            수입증감률.append(soup.findAll('td', {'align' : 'Right'})[i*10+8].text)
                            수지.append(soup.findAll('td', {'align' : 'Right'})[i*10+9].text)
                            i+=1

                        df = pd.DataFrame({'year' :년, 'month' :월, 'country':나라, 'SC':소분류, 'S_code':코드_소, 'export':수출액, 'export_rate':수출증감률, 'import':수입액, 'import_rate':수입증감률, 'CA':수지})
                        df_tot = pd.concat([df_tot, df])

                        #화살표 넘기기 
                        dr.find_element_by_xpath('''//*[@id="pageArea"]/a[2]''').click()

                except:
                    break;

2
3
14
15
16
17
18
19
20
22
23
24
25
26
27
28
29
30
32
33
34
35
36
47
48
49
2
3
4
5
6
7
8
9
10
12
13
14
15
16
17
18
19
20
22
23
24
25
26
27
28
29
30
32
33
34
35
36
37
38
39
40
42
43
44
45
46
47
48
49
50
41
43
44
45
46
47
48
49
50
41
43
44
45
46
47
48
49
50
41
43
44
45
46
47
48
49
50
41
43
44
45
46
47
48
49
50
41
43
44
45
46
47
48
49
50
41
43
44
45
46
47
48
49
50
41
43
44
45
46
47
48
49
50
41
43
44
45
46
47
48
49
50
41
43
44
45
46
47
48
49
50
41
43
44
45
46
47
48
49
50
41
43
44
45
46
47
48
49
50
41
43
44
45
46
47
48
49
50
41
43
44
45
46
47
48
49
50
41
43
44
45
46
47
48
49
50
41
43
44
45
46
47
48
49
50
41
43
44
45
46
47
48
49
50
41
43
44
45
46
47
48
49
50
41
43
44
45
46
47
48
49
50
41
43
44
45
46
47
48
49
50
41
43
44
45
46
47
48
49
50
41
43
44
45
46
47
48
49
50
41
43
44
45
46
47
48
49
50
41
43
44
45
46
47
48
49
50
41
43
44
45
46
47
48
49
50
41
43
44
45
46
47
48
49
50
41
43
44
45
46
47
48
49
50
41
43
44
45
46
47
48
49
50
41
43
44
45
46
47
48
49
50
41
43
44
45
46
47
48
49
50
41
43
44
45
46